# Getting Albums for All Rappers

In [11]:
import requests
from utils import Album, remove_duplicates, get_request_headers
import sqlite3

request_headers = get_request_headers()
db_con = sqlite3.connect('../albums.db')

Get list of rappers from `rappers.txt`.

In [2]:
with open('rappers.txt', 'r') as rappers_file:
    rappers_list = rappers_file.readlines()
    rappers_list = [rapper.strip() for rapper in rappers_list]

Get rapper ID on Spotify for each rapper.

In [3]:
SPOTIFY_SEARCH_URL = 'https://api.spotify.com/v1/search'

rapper_ids = []
for rapper in rappers_list:
    result = requests.get(SPOTIFY_SEARCH_URL, params={
        'q': rapper,
        'type': 'artist'
    }, headers=request_headers).json()

    rapper_info = result['artists']['items'][0]
    rapper_ids.append(rapper_info['id'])

Get all albums for each rapper and save them in a dictionary to take care of duplicates (e.g. collaborative albums).

In [4]:
SPOTIFY_ALBUMS_URL = 'https://api.spotify.com/v1/artists/{id}/albums?offset=0&limit=50&include_groups=album'
SPOTIFY_GET_ALBUMS_URL = 'https://api.spotify.com/v1/albums'

albums_map = {}
for rapper_id in rapper_ids:
    result = requests.get(SPOTIFY_ALBUMS_URL.format(id=rapper_id), headers=request_headers).json()
    album_ids = [a['id'] for a in result['items'] if a['album_type'] == 'album']
    for i in range(0, len(album_ids), 20):
        album_ids_upper_bound = i + 20 if (i + 20) < len(album_ids) else len(album_ids)
        result = requests.get(SPOTIFY_GET_ALBUMS_URL, params={'ids': ','.join(album_ids[i:album_ids_upper_bound])}, headers=request_headers).json()
        for a in result['albums']:
            if (not a['id'] in albums_map):
                tracks = a['tracks']['items']
                album_is_explicit = any([track['explicit'] for track in tracks])
                artist_names = [artist['name'] for artist in a['artists']]
                album_artists = ', '.join(artist_names)
                albums_map[a['id']] = Album(a['id'], a['name'], album_is_explicit, album_artists)

Save all albums in `albums.db` database file.

In [24]:
db_cursor = db_con.cursor()
db_cursor.execute(''' CREATE TABLE IF NOT EXISTS albums (
    spotify_id text PRIMARY KEY,
    name text NOT NULL,
    artists text NOT NULL,
    explicit integer NOT NULL    
); ''')

album_insert_statement = ''' INSERT INTO albums(spotify_id, name, artists, explicit)
VALUES (?,?,?,?); '''

final_albums_list = remove_duplicates(list(albums_map.values()))
for album in final_albums_list:
    db_cursor.execute(album_insert_statement, (album.id, album.name, album.artists, album.explicit))
db_con.commit()

In [26]:
db_con.close()